In [1]:
import sqlite3
conn = sqlite3.connect("datafile.db")
cursor = conn.cursor()#（電腦的）遊標
cursor

In [2]:
import sqlite3
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()
cursor

In [2]:
cursor.execute("""create table people(id integer
                  primary key, name text, count integer)""")
cursor.execute("""insert into people(name,count)
                   values("Bob",1)""")
username = "Jill"
usercount = 15
cursor.execute("""insert into people
    (name,count) values (?,?)""",(username,usercount))

conn.commit()

In [3]:
cursor.execute("""insert into people(name,count) values
               (:username,:usercount)""",{"username":"Joe","usercount":10})

In [6]:
result = cursor.execute("select * from people")
print(result.fetchall())#fetch:取回
result = cursor.execute("""select * from people
    where name like (:name)""",{"name":"Bob"})#感覺加()比較可讀
print(result.fetchall())
result = cursor.execute("""update people
    set count = ? where name = ?""",(20,"Jill"))
result = cursor.execute("select * from people")
result.fetchall()

[(1, 'Bob', 1), (2, 'Jill', 20), (3, 'Joe', 10)]
[(1, 'Bob', 1)]


[(1, 'Bob', 1), (2, 'Jill', 20), (3, 'Joe', 10)]

In [7]:
result = cursor.execute("select * from people")
for row in result:
    print(row)

(1, 'Bob', 1)
(2, 'Jill', 20)
(3, 'Joe', 10)


In [8]:
name = "Jill"
result = cursor.execute("""select * from people
    where name = '{}'""".format(name))
result.fetchall()

[(2, 'Jill', 20)]

In [12]:
bad_name = "1'or'1'='1"
result = cursor.execute("""select * from people
    where name = '{}'""".format(bad_name))
result.fetchall()

[(1, 'Bob', 1), (2, 'Jill', 20), (3, 'Joe', 10)]

In [14]:
bad_name = "1'or'1'='1"
#good_name = "Bob"
result = cursor.execute("""select * from people
    where name = (?)""",(bad_name,))
result.fetchall()

[]

In [15]:
cursor.execute("""update people set count = ?
    where name = ?""",(20,"Jill"))
conn.commit()
conn.close()

In [11]:
import sqlite3
conn = sqlite3.connect("test0.db")
cursor = conn.cursor()
cursor.execute("""create table weather0 (id integer primary key,
    a text,b text,
    c text,d text,e real,
    f integer,g real,
    h real,i real,
    j integer,k real,
    l real,m text)""")

conn.commit()

In [21]:
import csv
results = csv.reader(open("temp_data_01.csv",encoding = "utf-8"))
weather_list = []
for result in results:
    weather_list.append(result)
    
del weather_list[0]
print(weather_list)

[['', 'Illinois', '17', 'Jan 01, 1979', '1979/01/01', '17.48', '994', '6.00', '30.50', '2.89', '994', '-13.60', '15.80', 'Missing', '0', 'Missing', 'Missing', '0.00%'], ['', 'Illinois', '17', 'Jan 02, 1979', '1979/01/02', '4.64', '994', '-6.40', '15.80', '-9.03', '994', '-23.60', '6.60', 'Missing', '0', 'Missing', 'Missing', '0.00%'], ['', 'Illinois', '17', 'Jan 03, 1979', '1979/01/03', '11.05', '994', '-0.70', '24.70', '-2.17', '994', '-18.30', '12.90', 'Missing', '0', 'Missing', 'Missing', '0.00%'], ['', 'Illinois', '17', 'Jan 04, 1979', '1979/01/04', '9.51', '994', '0.20', '27.60', '-0.43', '994', '-16.30', '16.30', 'Missing', '0', 'Missing', 'Missing', '0.00%'], ['', 'Illinois', '17', 'May 15, 1979', '1979/05/15', '68.42', '994', '61.00', '75.10', '51.30', '994', '43.30', '57.00', 'Missing', '0', 'Missing', 'Missing', '0.00%'], ['', 'Illinois', '17', 'May 16, 1979', '1979/05/16', '70.29', '994', '63.40', '73.50', '48.09', '994', '41.10', '53.00', 'Missing', '0', 'Missing', 'Missing

In [27]:
import sqlite3
conn = sqlite3.connect("test.db")
cursor = conn.cursor()#創建表格時，不能用中文
cursor.execute("""create table weather (id integer primary key, 
state text, state_code text,
              year_text text, year_code text, avg_max_temp real,  
max_temp_count integer, 
              max_temp_low real, max_temp_high real,
              avg_min_temp real, min_temp_count integer,
              min_temp_low real, min_temp_high real,
              heat_index real, heat_index_count integer, 
              heat_index_low real, heat_index_high real,
              heat_index_coverage text)
              """)



conn.commit()

OperationalError: table weather already exists

In [ ]:
110570027
*Jelly198576

In [1]:
from sqlalchemy import create_engine,select,MetaData,Table,Column,Integer,String
from sqlalchemy.orm import sessionmaker

dbPath = "datafile2.db"
engine = create_engine("sqlite:///%s" % dbPath)
metadata = MetaData(engine)
people = Table("people",metadata,
              Column("id",Integer,primary_key = True),
              Column("name",String),
              Column("count",Integer))

Session = sessionmaker(bind = engine)
session = Session()
metadata.create_all(engine)

In [2]:
people_ins = people.insert().values(name = "Bob",count = 1)
str(people_ins)
session.execute(people_ins)
session.commit()

In [3]:
session.execute(people_ins,[
    {"name":"Jill","count":15},
    {"name":"Joe","count":10}
])
session.commit()
result = session.execute(select([people]))
for row in result:
    print(row)

(1, 'Bob', 1)
(2, 'Bob', 1)
(3, 'Bob', 1)
(4, 'Jill', 22)
(5, 'Joe', 10)
(6, 'Jill', 20)
(7, 'Joe', 10)
(8, 'Jill', 20)
(9, 'Joe', 10)
(10, 'Jill', 20)
(11, 'Joe', 10)
(12, 'Bob', 1)
(13, 'Jill', 15)
(14, 'Joe', 10)


In [4]:
result = session.execute(
    select([people]).where(people.c.name == "Jill"))
for row in result:
    print(row)

(4, 'Jill', 22)
(6, 'Jill', 20)
(8, 'Jill', 20)
(10, 'Jill', 20)
(13, 'Jill', 15)


In [12]:
result = session.execute(
    people.update().values(count = 20).where(
    people.c.name == "Jill"))
session.commit()
result = session.execute(
    select([people]).where(people.c.name == "Jill"))
for row in result:
    print(row)

(4, 'Jill', 20)
(6, 'Jill', 20)
(8, 'Jill', 20)
(10, 'Jill', 20)


In [13]:
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
class People(Base):
    __tablename__ = "people"
    id = Column(Integer,primary_key = True)
    name = Column(String)
    count = Column(Integer)
    
results = session.query(People).filter_by(name = "Jill")
for person in results:
    print(person.id,person.name,person.count)

4 Jill 20
6 Jill 20
8 Jill 20
10 Jill 20


In [14]:
new_person = People(name = "Jane",count = 5)
session.add(new_person)
session.commit()
results = session.query(People).all()
for person in results:
    print(person.id,person.name,person.count)

1 Bob 1
2 Bob 1
3 Bob 1
4 Jill 20
5 Joe 10
6 Jill 20
7 Joe 10
8 Jill 20
9 Joe 10
10 Jill 20
11 Joe 10
12 Jane 5


In [18]:
jill = session.query(People).filter_by(name = "Jill").first()
print(jill.name,jill.count)
jill.count = 22
session.add(jill)
session.commit()
results = session.query(People).all()
for person in results:
    print(person.id,person.name,person.count)

Jill 20
1 Bob 1
2 Bob 1
3 Bob 1
4 Jill 22
5 Joe 10
6 Jill 20
7 Joe 10
8 Jill 20
9 Joe 10
10 Jill 20
11 Joe 10
12 Jane 5


In [25]:
jane = session.query(People).filter_by(name = "Jane").first()
#print(jane)
#session.delete(jane)
session.commit()
jane = session.query(People).filter_by(name = "Jane").first()
print(jane)

None


In [4]:
from sqlalchemy import create_engine, select, MetaData, Table, Column, Integer, String, Float
from sqlalchemy.orm import sessionmaker

dbPath = 'datafile.db'
engine = create_engine('sqlite:///%s' % dbPath)
metadata = MetaData(engine)
weather  = Table('weather', metadata, 
                Column('id', Integer, primary_key=True),
                Column("state", String),
                Column("state_code", String),
                Column("year_text", String ),
                Column("year_code", String), 
                Column("avg_max_temp", Float),
                Column("max_temp_count", Integer),
                Column("max_temp_low", Float),
                Column("max_temp_high", Float),
                Column("avg_min_temp", Float), 
                Column("min_temp_count", Integer),
                Column("min_temp_low", Float), 
                Column("min_temp_high", Float),
                Column("heat_index", Float), 
                Column("heat_index_count", Integer),
                Column("heat_index_low", Float), 
                Column("heat_index_high", Float),
                Column("heat_index_coverage", String)
                )
Session = sessionmaker(bind=engine)
session = Session()
result = session.execute(select([weather]))
for row in result:
    print(row)

In [3]:
print(engine.table_names())

['people']


C:\Users\Vick\AppData\Local\Temp/ipykernel_18868/1043810782.py:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  print(engine.table_names())


In [5]:
engine.table_names()

C:\Users\Vick\AppData\Local\Temp/ipykernel_18868/4162273999.py:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['address', 'alembic_version', 'people']

In [6]:
engine.table_names()

C:\Users\Vick\AppData\Local\Temp/ipykernel_18868/4162273999.py:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['alembic_version', 'people']

In [7]:
import redis
r = redis.Redis(host = "localhost",port = 6379)

In [8]:
r.keys()

ConnectionError: Error 10061 connecting to localhost:6379. 無法連線，因為目標電腦拒絕連線。.

In [7]:
import pymongo

ModuleNotFoundError: No module named 'pymongo'